In [ ]:
import os
import pickle
import paramiko
from scp import SCPClient
import tkinter as tk
import speech_recognition as sr
from tkinter import ttk, messagebox
from mistralai import Mistral
import whisper  
import threading
import numpy as np
import torch
import torchaudio
import time
import wave
import contextlib
import sys


print(sys.executable)
print("✅ pydub is working!")


whisper_model = whisper.load_model("large-v3.pt")

# **API Configuration**
api_key = "****"
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

# **Storage Path**
output_folder = r"results"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "mistral_result.pkl")


# **Ubuntu SSH Information**
ubuntu_host = "192.168.137.***"  
ubuntu_user = "jetson"           
ubuntu_password = "***"  
remote_file = "/home/jetson/notebooks/mistral_result.pkl"  

# **Upload to Ubuntu**
def upload_to_ubuntu(local_file, remote_file):
    try:
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(ubuntu_host, username=ubuntu_user, password=ubuntu_password)

        scp = SCPClient(ssh.get_transport())
        scp.put(local_file, remote_file)
        scp.close()
        ssh.close()

        messagebox.showinfo("Success", f"✅ Result uploaded to: {remote_file}")

    except Exception as e:
        messagebox.showerror("Upload Failed", f"❌ Upload error: {e}")

# **Create Window**
root = tk.Tk()
root.title("Color Choice Game")
root.geometry("800x500")
root.configure(bg="white")

# **Keep Window Centered**
def center_window(window):
    window.update_idletasks()
    w = window.winfo_width()
    h = window.winfo_height()
    screen_w = window.winfo_screenwidth()
    screen_h = window.winfo_screenheight()
    x = (screen_w - w) // 2
    y = (screen_h - h) // 2
    window.geometry(f"{w}x{h}+{x}+{y}")

# **Color Palette**
color_palette = [
    "#0096A5", "#005066", "#E63946",
    "#F77F00", "#E6399B", "#457B9D"
]

# **Top Logo**
logo_text = "Color Choice Game"
logo_frame = tk.Frame(root, bg="white")
logo_frame.pack(pady=(50, 20))

for i, letter in enumerate(logo_text):
    color = color_palette[i % len(color_palette)]
    label = tk.Label(logo_frame, text=letter, font=("Arial", 38, "bold"), fg=color, bg="white")
    label.pack(side="left")

# **Search Box**
search_width = 600
search_frame = tk.Frame(root, bg="white", highlightthickness=2, highlightbackground="#ddd", relief="solid")
search_frame.place(relx=0.5, rely=0.4, anchor="center", width=search_width, height=80)

entry = tk.Text(search_frame, font=("Arial", 18), bd=0, wrap="word", height=2)
entry.pack(fill="both", expand=True, padx=20, pady=10)



def open_recording_window():
    """ recording window """
    global record_window
    record_window = tk.Toplevel(root)
    record_window.title("🎤 Recording")
    record_window.geometry("400x200")
    record_window.configure(bg="white")
    record_window.transient(root)  
    record_window.grab_set()  
    center_window(record_window)  
    

    status_label = tk.Label(record_window, text="🎤 Recording...", font=("Arial", 14), fg="black", bg="white")
    status_label.pack(pady=20)


    stop_button = tk.Button(record_window, text="🛑 Finished", font=("Arial", 14), bg="red", fg="white",
                            command=stop_recording)
    stop_button.pack(pady=10, padx=20, ipadx=10, ipady=5, fill="x", expand=True)


    record_window.update_idletasks()
    record_window.update()


    threading.Thread(target=recognize_speech, daemon=True).start()


def stop_recording():
    if record_window:
        record_window.destroy()  # closed



# **API Request Function**
def get_mistral_color():
    user_input = entry.get("1.0", tk.END).strip()
    
    if not user_input:
        messagebox.showwarning("Warning", "Please enter.")
        return
    
    result_label.config(text="⏳ AI is thinking...", fg="gray")
    root.update()

    try:
        chat_response = client.chat.complete(
            model=model,
            messages=[{"role": "user", "content": user_input+"only answer the color name,thinking what color I want,have to be red, blue, green, yellow. the answer can be one or more.The format is [color,color]."}]
        )
       
        ai_result = chat_response.choices[0].message.content.strip()
        result_label.config(text=f"🎯 Result: {ai_result}", fg="black")
        
        # Save Result
        with open(output_file, "wb") as f:
            pickle.dump(ai_result, f)
        
        messagebox.showinfo("Success", f"Result saved to: {output_file}")

        # **Upload to Ubuntu after saving**
        upload_to_ubuntu(output_file, remote_file)
    
    except Exception as e:
        messagebox.showerror("Error", f"API request failed: {e}")



print("11111111111")


# **Search Button**
search_button = tk.Button(root, text="Search", font=("Arial", 14), bg="#0096A5", fg="white",
                          relief="flat", padx=10, pady=5, command=get_mistral_color)
search_button.place(relx=0.5, rely=0.52, anchor="center", width=180, height=50)

# **Result Box**
result_frame = tk.Frame(root, bg="white", highlightthickness=2, highlightbackground="#ddd", relief="solid")
result_frame.place(relx=0.5, rely=0.65, anchor="center", width=600, height=50)

result_label = tk.Label(result_frame, text="Result will appear here...", font=("Arial", 14), fg="gray", bg="white")
result_label.pack(fill="both", expand=True, padx=10, pady=5)
print("2222222222222222")

def recognize_speech():
    with sr.Microphone() as source:
        messagebox.showinfo("Listening", "🎤 Speak now... ")
        recognizer = sr.Recognizer()
        recognizer.adjust_for_ambient_noise(source)
        
        try:
            
            audio = recognizer.listen(source, timeout=20, phrase_time_limit=20)
            
            # **save to the local WAV file**
            wav_path = os.path.abspath(r"temp_audio.wav")
            with open(wav_path, "wb") as f:
                f.write(audio.get_wav_data())

            print(f"✅ Recording saving: {wav_path}")

            # **Whisper voice recognazation**
            result = whisper_model.transcribe(wav_path)
            recognized_text = result["text"].strip().lower()
            print("Transcribed Text:\n", recognized_text)

            
            # **API abstrac color**
            chat_response = client.chat.complete(
                model=model,
                messages=[{"role": "user", "content": recognized_text + 
                           " 只回答颜色名称,判断我想要的颜色,只能是红色，蓝色，绿色，黄色这四个词的英文，答案可以是一个或者多个，格式为 [颜色,颜色]。"}])

            # **got AI returned the file**
            extracted_colors = chat_response.choices[0].message.content.strip().lower()

    
            extracted_colors = extracted_colors.replace("[", "").replace("]", "").replace("'", "").replace('"', "").split(",")

 
            valid_colors = {"red", "blue", "green", "yellow"}
            extracted_color = [color.strip() for color in extracted_colors if color.strip() in valid_colors]
#             extracted_color = chat_response.choices[0].message.content.strip().lower()
#             valid_colors = {"red", "blue", "green", "yellow"}
            
            # **If no color is matched, return ["unknown"]**
            if not extracted_color:
                extracted_color = ["unknown"]

            print(f"🎨 Grab color: {extracted_color}")

            # **saving color outcome**
            with open(output_file, "wb") as f:
                pickle.dump(extracted_color, f)

            messagebox.showinfo("Success", f"Speech recognition saved to: {output_file}")

            # **upload to Ubuntu**
            upload_to_ubuntu(output_file, remote_file)

        except sr.WaitTimeoutError:
            messagebox.showerror("Timeout", "No speech detected.")
        except FileNotFoundError as fe:
            messagebox.showerror("Error", f"Audio file not found: {fe}")
        except Exception as e:
            messagebox.showerror("Error", f"Speech recognition failed: {e}")




# **Microphone Button**

mic_button = tk.Button(root, text="🎤", font=("Arial", 18), bg="#0096A5", fg="white", relief="flat", command=open_recording_window)
mic_button.place(relx=0.85, rely=0.4, anchor="center", width=50, height=50)
print("333333333333")

# **Run GUI**
root.mainloop()

C:\Users\T2Chen\.conda\envs\robot\python.exe
✅ pydub is working!
11111111111
2222222222222222
333333333333


In [4]:
999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999*9999999999999999999999999999999999999999999999999999999999999999999999999

9999999999999999999999999999999999999999999999999999999999999999999999998999999999999999990000000000000000000000000000000000000000000000000000000000000000000000001